In [ ]:
from foobah.gcode import GCODE
from foobah.utils import preview_svg
from foobah.methods import line_threshold
from foobah import constants
from foobah.constants import XMAX, XMIN, YMAX, YMIN

import PIL
import PIL.Image
import PIL.ImageFilter

import numpy as np

from random import uniform

from IPython.display import display

In [ ]:
basename = "pixel_david_square_wave"

scale = 1
image_name = "images/square.png"
image_name = "images/david-bowie.jpg"
image = PIL.Image.open(image_name)

image = image.resize((int(image.width / scale), int(image.height / scale)), PIL.Image.ANTIALIAS)
# image = image.filter(PIL.ImageFilter.EDGE_ENHANCE_MORE)
# image = image.filter(PIL.ImageFilter.SMOOTH_MORE)
# image = image.filter(PIL.ImageFilter.CONTOUR)
# image = image.filter(PIL.ImageFilter.DETAIL)
# image = image.filter(PIL.ImageFilter.EDGE_ENHANCE)
# image = image.filter(PIL.ImageFilter.EDGE_ENHANCE_MORE)
# image = image.filter(PIL.ImageFilter.SMOOTH_MORE)

# image = image.rotate(-90, expand=True)

image = image.convert("RGB")

width = image.width
height = image.height

display(image)

In [ ]:
def move_to(gcode, x, y):
    gcode.move_to(
        ((x) / image.width) * (XMAX - XMIN) + XMIN,
        ((y) / image.height) * (YMAX - YMIN) + YMIN,
    )
    

def draw_square(x, y, pixel_size=10):
    gcode.pen_up()
    
    gcode.move_to(
        ((x - pixel_size / 2.0) / image.width) * (XMAX - XMIN) + XMIN,
        ((y - pixel_size / 2.0) / image.height) * (YMAX - YMIN) + YMIN,
    )
    
    gcode.pen_down()
    
    gcode.move_to(
        ((x + pixel_size / 2.0) / image.width) * (XMAX - XMIN) + XMIN,
        ((y - pixel_size / 2.0) / image.height) * (YMAX - YMIN) + YMIN,
    )
    gcode.move_to(
        ((x + pixel_size / 2.0) / image.width) * (XMAX - XMIN) + XMIN,
        ((y + pixel_size / 2.0) / image.height) * (YMAX - YMIN) + YMIN,
    )
    gcode.move_to(
        ((x - pixel_size / 2.0) / image.width) * (XMAX - XMIN) + XMIN,
        ((y + pixel_size / 2.0) / image.height) * (YMAX - YMIN) + YMIN,
    )
    gcode.move_to(
        ((x - pixel_size / 2.0) / image.width) * (XMAX - XMIN) + XMIN,
        ((y - pixel_size / 2.0) / image.height) * (YMAX - YMIN) + YMIN,
    )
    gcode.pen_up()
    
    
def fill_square(x_center, y_center, pixel_size=10, dy=2, min_dy=0.5, zigzag=True):
    dy = max(dy, min_dy)
        
    gcode.pen_up()
    
    gcode.move_to(
        ((x_center - pixel_size / 2.0) / image.width) * (XMAX - XMIN) + XMIN,
        ((y_center - pixel_size / 2.0) / image.height) * (YMAX - YMIN) + YMIN,
    )
    
    gcode.pen_down()
    
    y = y_center - pixel_size / 2.0
    counter = 0
    
    while y < y_center + pixel_size / 2.0:
        if counter % 2 == 0:
            x = x_center + pixel_size / 2.0
        else:
            x = x_center - pixel_size / 2.0
        
        if not zigzag:
            move_to(gcode, x, y)
        
        y += dy
        
        move_to(gcode, x, y)
        
        counter += 1
        
    gcode.pen_up()


def pixel_block(x_center, y_center, pixel_size=10, dy=2, zigzag=True):
#     draw_square(x_center, y_center, pixel_size=pixel_size)
    fill_square(x_center, y_center, pixel_size=pixel_size, dy=dy, zigzag=zigzag)

In [ ]:
gcode = GCODE(basename, feedrate=3500)
gcode.move_to_starting_position()
gcode.pen_down()
gcode.draw_boundaries()
gcode.pen_up()

gcode.move_to_mid_point()
gcode.pen_up()

In [ ]:
from math import sqrt, pow

pixel_size = 8
max_dy = 4

for x in range(0, image.width, pixel_size):
    for y in range(0, image.height, pixel_size):
        intensity = 0
        for x_ in range(0, pixel_size):
            for y_ in range(0, pixel_size):
                if x + x_ >= image.width or y + y_ >= image.height:
                    continue
                r, g, b = image.getpixel((x + x_, y + y_))
                intensity += ((255 - r) + (255 - g) + (255 - b)) / (255 * 3.0)
                
        intensity /= pixel_size * pixel_size
        
#         r, g, b = image.getpixel((x, y))
#         intensity = ((255 - r) + (255 - g) + (255 - b)) / (255 * 3.0)
        
        if intensity > 0.025:
#             pixel_intensity = 3.0 - ((pow(intensity, 2) * 1) * 3.0)
            pixel_intensity = max_dy - ((pow(intensity, 1 / 1.5) * 1) * max_dy)
#             pixel_intensity = 3.0 - ((pow(intensity, 1/2) * 1) * 3.0)
#             pixel_intensity = 3.0 - ((sqrt(intensity) * 1) * 3.0)
            pixel_block(x - pixel_size / 2.0, y - pixel_size / 2.0, pixel_size=pixel_size, dy=pixel_intensity, zigzag=False)

In [ ]:
# pixel_block(100, 100, pixel_size=20, dy=2, zigzag=True)
# pixel_block(120, 100, pixel_size=20, dy=2, zigzag=False)

In [ ]:
gcode.pen_up()
gcode.move_to_starting_position()
gcode.flush()

In [ ]:
preview_svg(basename)